In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
import requests

@tool
def search_youtube_video(query: str, region: str = "US", language: str = "en") -> dict:
    """
    Search for a video on YouTube using the RapidAPI YouTube138 API.

    Args:
        query (str): The search query (e.g., video title or keyword).
        region (str): The region code (default is 'US').
        language (str): The language code (default is 'en').

    Returns:
        dict: JSON containing details of the top video result.
    """
    url = "https://youtube138.p.rapidapi.com/search/"
    params = {
        "q": query,
        "hl": language,
        "gl": region
    }
    headers = {
        "x-rapidapi-key": "api_key_here",  # Replace with your RapidAPI key
        "x-rapidapi-host": "youtube138.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        for result in data.get("contents", []):
            video = result.get("video")
            if video:
                video_id = video.get("videoId")
                return {
                    "query": query,
                    "video_id": video_id,
                    "video_url": f"https://www.youtube.com/watch?v={video_id}",
                    "title": video.get("title"),
                    "description": video.get("descriptionSnippet"),
                    "duration_seconds": video.get("lengthSeconds"),
                    "published_time": video.get("publishedTimeText"),
                    "views": video.get("stats", {}).get("views"),
                    "thumbnail": video.get("thumbnails", [{}])[0].get("url"),
                    "channel": {
                        "title": video.get("author", {}).get("title"),
                        "channel_id": video.get("author", {}).get("channelId"),
                        "avatar": video.get("author", {}).get("avatar", [{}])[0].get("url"),
                        "channel_url": f"https://www.youtube.com/channel/{video.get('author', {}).get('channelId')}"
                    }
                }

        return {"message": "No video found for your query."}

    except requests.exceptions.RequestException as e:
        return {"error": f"Error during YouTube search: {str(e)}"}
    except (KeyError, IndexError, TypeError) as e:
        return {"error": f"Unexpected response format: {str(e)}"}

 

In [ ]:
# youtube_assistant = create_react_agent(
#     model=model,
#     tools=[search_youtube_video],
#     prompt="You are a helpful YouTube video assistant. Your job is to help users find YouTube videos based on their search queries.",
#     name="youtube_assistant"
# )
youtube_assistant = create_react_agent(
    model=model,
    tools=[search_youtube_video],
    prompt=(
        "You are a helpful YouTube video assistant. Your job is to help users find YouTube videos "
        "based on their search queries using the `search_youtube_video` tool.\n\n"
        "🧠 Think step-by-step and always call the tool using the user's query.\n"
        "📦 When returning the result to the user, **always format the output strictly as JSON** using the structure below:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "video_id": string,\n'
        '  "video_url": string,\n'
        '  "title": string,\n'
        '  "description": string,\n'
        '  "duration_seconds": integer,\n'
        '  "published_time": string,\n'
        '  "views": integer,\n'
        '  "thumbnail": string,\n'
        '  "channel": {\n'
        '    "title": string,\n'
        '    "channel_id": string,\n'
        '    "avatar": string,\n'
        '    "channel_url": string\n'
        "  }\n"
        "}\n\n"
        "❗️Do not include any commentary or explanation outside the JSON. Only return valid JSON."
    ),
    name="youtube_assistant"
)


In [ ]:
response = youtube_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="5 minute video of yoga.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='5 minute video of yoga.', additional_kwargs={}, response_metadata={}, id='f792c920-0500-44ea-9257-ef89d9bea1a8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vJfq7nGV2TS7jxEmwZ1uKrdu', 'function': {'arguments': '{"query":"5 minute yoga"}', 'name': 'search_youtube_video'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 329, 'total_tokens': 347, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjJS2QFtI8t1XvnEZ8wAfKtSPBR1T', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='youtube_assistant', id='run--965a1a97-f3cf-432b-8e5a-3ba2bd4d37f0-0', tool_calls=[{'name': 'search_youtube_video', 'arg

In [ ]:
response["messages"][-1].content

'{\n  "query": "5 minute yoga",\n  "video_id": "p39GdEUxxAk",\n  "video_url": "https://www.youtube.com/watch?v=p39GdEUxxAk",\n  "title": "5 Minute Yoga for Back Tension (6 Movements of the Spine)",\n  "description": "Do this 5 minute yoga class to help relieve back tension and general back pain. This quick yoga class incorporated the 6\xa0...",\n  "duration_seconds": 309,\n  "published_time": "4 years ago",\n  "views": 284627,\n  "thumbnail": "https://i.ytimg.com/vi/p39GdEUxxAk/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLD3LpOvkKA1FW76c-ouuy4aSyJVdg",\n  "channel": {\n    "title": "Yoga With Bird",\n    "channel_id": "UCxYVPua6HC46HzgT8IIn3vg",\n    "avatar": "https://yt3.ggpht.com/i9LIxpbZvG6-57SHsq7tio9cbp109r1USkpeFuexB9fYK-En-50r4V7Jn2Vp9nsh_HNUql-vng=s68-c-k-c0x00ffffff-no-rj",\n    "channel_url": "https://www.youtube.com/channel/UCxYVPua6HC46HzgT8IIn3vg"\n  }\n}'

# Class

In [ ]:
import requests
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["RAPIDAPI_KEY"] = userdata.get("RAPIDAPI_KEY")
load_dotenv()

class YoutubeAgent:
    def __init__(self):
        self.openai_api_key = os.getenv('OPENAI_API_KEY')
        self.rapidapi_key = os.getenv('RAPIDAPI_KEY') # rapidapi_key or "api key here"  # Replace with your RapidAPI key
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,

            tools=[self.search_youtube_video],
            prompt=(
    "You are a helpful YouTube video assistant. Your job is to help users find YouTube videos "
    "based on their search queries. Use the tool to perform a search and return the top result. "
    "Once the top result is found, return it as 'Final Answer: <result>' and stop."
),
            name="youtube_assistant"
        )
        # print(self.openai_api_key)
        # print(self.rapidapi_key)

    @tool
    def search_youtube_video(self, query: str, region: str = "US", language: str = "en") -> str:
        """
        Search for a video on YouTube using the RapidAPI YouTube138 API.
        Args:
            query (str): The search query (e.g., video title or keyword).
            region (str): The region code (default is 'US').
            language (str): The language code (default is 'en').
        Returns:
            str: The title and URL of the top video result.
        """
        print("*********************Tool Invoked")
        url = "https://youtube138.p.rapidapi.com/search/"
        params = {
            "q": query,
            "hl": language,
            "gl": region
        }
        headers = {
            "x-rapidapi-key": self.rapidapi_key,
            "x-rapidapi-host": "youtube138.p.rapidapi.com"
        }
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            for result in data.get("contents", []):
                video = result.get("video")
                if video:
                    print("Hello")
                    title = video.get("title")
                    print("Hello", title)
                    video_id = video.get("videoId")
                    print("Hello", video_id)
                    video_url = f"https://www.youtube.com/watch?v={video_id}"
                    return f"Final Answer: Top result:\nTitle: {title}\nURL: {video_url}"
            return "Final Answer: No video found for your query."
        except requests.exceptions.RequestException as e:
            return f"Error during YouTube search: {str(e)}"
        except (KeyError, IndexError, TypeError) as e:
            return f"Unexpected response format: {str(e)}"

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        },
        config={"recursion_limit": 50})
        return response["messages"][-1].content


In [ ]:
# Usage example for YoutubeAgent class
youtube_agent = YoutubeAgent()
result = youtube_agent.run("5 minute yoga video")
print(result)

Sorry, need more steps to process this request.


In [ ]:
# Usage example for YoutubeAgent class
youtube_agent = YoutubeAgent()
result = youtube_agent.run("5 minute yoga video")
print(result)

I'm having trouble retrieving the requested information at the moment. Please try again later.
